In [1]:
import sys
import os
import tensorflow as tf 
import numpy as np
import time

In [2]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
def maxpool2d(x, kernel_size=3,strides=1):
    return tf.nn.max_pool(x, ksize=[1, kernel_size, kernel_size, 1], 
                          strides=[1, strides, strides, 1],padding='SAME')
def deconv2d(x,W,output_shape,strides=1):
    return tf.nn.conv2d_transpose(x,W,output_shape,strides=[1,strides,strides,1],padding='SAME')
def print_tensor_shape(tensor,string):
    print(string,tensor.get_shape())

In [3]:
# Store layers weight & bias
weights = {
    # 3x3 conv, 4 input, 64 outputs
    'w_s1': tf.Variable(tf.random_normal([3, 3, 4, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s2': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 7x7 conv, 4 input, 128 outputs
    'w_m1': tf.Variable(tf.random_normal([7, 7, 4, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m2': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    # 11x11 conv, 4 input, 256 outputs
    'w_l1': tf.Variable(tf.random_normal([11,11, 4, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l2': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    # 1x1 conv, 448 inputs, 300 outputs
    'w_f1': tf.Variable(tf.random_normal([1, 1, 448, 300])),
    # 1x1 conv, 300 inputs, 512 outputs
    'w_f2': tf.Variable(tf.random_normal([1, 1, 300, 512])),
    # 1x1 conv, 512 inputs, 5 outputs
    'w_f3': tf.Variable(tf.random_normal([1, 1, 512, 5])),
    # 3x3 conv, 448 inputs, 300 outputs
    'w_f4': tf.Variable(tf.random_normal([3, 3, 448, 300])),
    # 1x1 conv, 5 inputs, 5 outputs
    'w_fdec1': tf.Variable(tf.random_normal([1, 1, 5, 5])),
    # 1x1 conv, 10 inputs, 5 outputs
    'w_final_fusion': tf.Variable(tf.random_normal([1, 1, 10, 5])),
    
}


biases = {
    'b_s': tf.Variable(tf.random_normal([64])),
    'b_m': tf.Variable(tf.random_normal([128])),
    'b_l': tf.Variable(tf.random_normal([256])),
    'b_f1': tf.Variable(tf.random_normal([300])),
    'b_f2': tf.Variable(tf.random_normal([512])),
    'b_f3': tf.Variable(tf.random_normal([5])),
    'b_ff': tf.Variable(tf.random_normal([5]))
}
dropout = 0.5

In [4]:
def net(x,weights,biases,dropout):
    print_tensor_shape(x,'X antes del Reshaping')
    x = tf.reshape(x, shape=[-1, 240, 240, 4])
    #x = tf.reshape(x,[batch_size,240,240,-1])
    print_tensor_shape(x,'X despues del Reshaping')
    print("\n")
    #WHERE
    #SMALL
    conv1 = conv2d(x,weights['w_s1'],biases['b_s'],strides=1)
    print_tensor_shape(conv1,'Shape conv1')
    pool1 = maxpool2d(conv1,kernel_size=3,strides=1)
    print_tensor_shape(pool1,'Shape pool1')
    conv2 = conv2d(pool1,weights['w_s2'],biases['b_s'],strides=1)
    print_tensor_shape(conv2,'Shape conv2')
    pool2 = maxpool2d(conv2,kernel_size=3,strides=1)
    print_tensor_shape(pool2,'Shape pool2')
    conv3 = conv2d(pool2,weights['w_s2'],biases['b_s'],strides=1)
    print_tensor_shape(conv3,'Shape conv3')
    pool3 = maxpool2d(conv3,kernel_size=3,strides=1)
    print_tensor_shape(pool3,'Shape pool3')
    conv4 = conv2d(pool3,weights['w_s2'],biases['b_s'],strides=1)
    print_tensor_shape(conv4,'Shape conv4')
    pool4 = maxpool2d(conv4,kernel_size=3,strides=1)
    print_tensor_shape(pool4,'Shape pool4')
    conv5 = conv2d(pool4,weights['w_s2'],biases['b_s'],strides=1)
    print_tensor_shape(conv5,'Shape conv5')
    pool5 = maxpool2d(conv5,kernel_size=3,strides=1)
    print_tensor_shape(pool5,'Shape pool5')
    print("\n")
    #MEDIUM
    conv6 = conv2d(x,weights['w_m1'],biases['b_m'],strides=1)
    print_tensor_shape(conv6,'Shape conv6')
    pool6 = maxpool2d(conv6,kernel_size=3,strides=1)
    print_tensor_shape(pool6,'Shape pool4')
    conv7 = conv2d(pool6,weights['w_m2'],biases['b_m'],strides=1)
    print_tensor_shape(conv7,'Shape conv7')
    pool7 = maxpool2d(conv7,kernel_size=3,strides=1)
    print_tensor_shape(pool7,'Shape pool7')
    conv8 = conv2d(pool7,weights['w_m2'],biases['b_m'],strides=1)
    print_tensor_shape(conv8,'Shape conv8')
    pool8 = maxpool2d(conv8,kernel_size=3,strides=1)
    print_tensor_shape(pool8,'Shape pool8')
    conv9 = conv2d(pool8,weights['w_m2'],biases['b_m'],strides=1)
    print_tensor_shape(conv9,'Shape conv9')
    pool9 = maxpool2d(conv9,kernel_size=3,strides=1)
    print_tensor_shape(pool9,'Shape pool9')
    conv10 = conv2d(pool9,weights['w_m2'],biases['b_m'],strides=1)
    print_tensor_shape(conv10,'Shape conv10')
    pool10 = maxpool2d(conv10,kernel_size=3,strides=1)
    print_tensor_shape(pool10,'Shape pool10')
    print("\n")
    #LARGE
    conv11 = conv2d(x,weights['w_l1'],biases['b_l'],strides=1)
    print_tensor_shape(conv11,'Shape conv11')
    pool11 = maxpool2d(conv11,kernel_size=3,strides=1)
    print_tensor_shape(pool11,'Shape pool11')
    conv12 = conv2d(pool11,weights['w_l2'],biases['b_l'],strides=1)
    print_tensor_shape(conv12,'Shape conv12')
    pool12 = maxpool2d(conv12,kernel_size=3,strides=1)
    print_tensor_shape(pool12,'Shape pool12')
    conv13 = conv2d(pool12,weights['w_l2'],biases['b_l'],strides=1)
    print_tensor_shape(conv13,'Shape conv13')
    pool13 = maxpool2d(conv13,kernel_size=3,strides=1)
    print_tensor_shape(pool11,'Shape pool13')
    conv14 = conv2d(pool13,weights['w_l2'],biases['b_l'],strides=1)
    print_tensor_shape(conv12,'Shape conv14')
    pool14 = maxpool2d(conv14,kernel_size=3,strides=1)
    print_tensor_shape(pool14,'Shape pool14')
    conv15 = conv2d(pool14,weights['w_l2'],biases['b_l'],strides=1)
    print_tensor_shape(conv15,'Shape conv15')
    pool15 = maxpool2d(conv15,kernel_size=3,strides=1)
    print_tensor_shape(pool14,'Shape pool15')
    print("\n")
    #FUSION
    conv16 = tf.concat((pool5,pool10,pool15),3)
    print_tensor_shape(conv16,'Shape conv16_concat')
    conv16 = conv2d(conv16,weights['w_f1'],biases['b_f1'],strides=1)
    print_tensor_shape(conv16,'Shape conv16')
    drop1 = tf.nn.dropout(conv16,dropout)
    print_tensor_shape(drop1,'Shape drop1')
    conv17 = conv2d(drop1,weights['w_f2'],biases['b_f2'],strides=1)
    print_tensor_shape(conv17,'Shape conv17')
    drop2 = tf.nn.dropout(conv17,dropout)
    print_tensor_shape(drop2,'Shape drop2')
    conv18 = conv2d(drop2,weights['w_f3'],biases['b_f3'],strides=1)
    print_tensor_shape(conv18,'Shape conv18')
    print("\n")
    #WHAT
    #LARGE
    conv1l =conv2d(x,weights['w_l1'],biases['b_l'],strides=4)
    print_tensor_shape(conv1l,'Shape conv1l')
    pool1l= maxpool2d(conv1l,kernel_size=2,strides=2)
    print_tensor_shape(pool1l,'Shape pool1')
    print("\n")
    #MEDIUM
    conv1m =conv2d(x,weights['w_m1'],biases['b_m'],strides=2)
    print_tensor_shape(conv1m,'Shape conv1m')
    pool1m= maxpool2d(conv1m,kernel_size=2,strides=2)
    print_tensor_shape(pool1m,'Shape pool1m')
    conv2m = conv2d(pool1m,weights['w_m2'],biases['b_m'],strides=2)
    print_tensor_shape(conv2m,'Shape conv2m')
    print("\n")
    #SMALL
    conv1s =conv2d(x,weights['w_s1'],biases['b_s'],strides=1)
    print_tensor_shape(conv1s,'Shape conv1s')
    pool1s= maxpool2d(conv1s,kernel_size=2,strides=2)
    print_tensor_shape(pool1s,'Shape pool1s')
    conv2s = conv2d(pool1s,weights['w_s2'],biases['b_s'],strides=1)
    print_tensor_shape(conv2s,'Shape conv2s')
    pool2s= maxpool2d(conv2s,kernel_size=2,strides=2)
    print_tensor_shape(pool2s,'Shape pool2s')
    conv3s = conv2d(pool2s,weights['w_s2'],biases['b_s'],strides=1)
    print_tensor_shape(conv3s,'Shape conv3s')
    pool3s= maxpool2d(conv3s,kernel_size=2,strides=2)
    print_tensor_shape(pool3s,'Shape pool3s')
    conv4s = conv2d(pool3s,weights['w_s2'],biases['b_s'],strides=1)
    print_tensor_shape(conv4s,'Shape conv3s')
    print("\n")
    #FUSION
    convf1 = tf.concat((pool1l,conv2m,conv4s),3) 
    print_tensor_shape(convf1,'Shape convf1')
    convf2 = conv2d(convf1,weights['w_f4'],biases['b_f1'],strides=1)
    print_tensor_shape(convf2,'Shape convf2')
    poolf2= maxpool2d(convf2,kernel_size=2,strides=2)
    print_tensor_shape(poolf2,'Shape poolf2')
    dropf3 = tf.nn.dropout(poolf2,dropout)
    print_tensor_shape(dropf3,'Shape dropf3')
    convf4 = conv2d(dropf3,weights['w_f2'],biases['b_f2'],strides=1)
    print_tensor_shape(convf4,'Shape convf4')
    dropf5 = tf.nn.dropout(convf4,dropout)
    print_tensor_shape(dropf5,'Shape dropf5')
    convf5 = conv2d(dropf5,weights['w_f3'],biases['b_f3'],strides=1)
    print_tensor_shape(convf5,'Shape convf5')
    print("\n")
    #DECONV
    deconv1= deconv2d(convf5,weights['w_fdec1'],output_shape = [-1,60,60,5],strides=4)
    print_tensor_shape(deconv1,'Shape deconv1')
    deconv2 = deconv2d(deconv1,weights['w_fdec1'],output_shape = [-1,240,240,5],strides=4)
    print_tensor_shape(deconv2,'Shape deconv2')
    print("\n")
    #FINAL FUSION
    conv_final_fusion1 = tf.concat((conv18,deconv2),3)
    print_tensor_shape(conv_final_fusion1,'Shape conv_final_fusion1')
    conv_final_fusion2 = conv2d(conv_final_fusion1,weights['w_final_fusion'],biases['b_ff'],strides=1)
    print_tensor_shape(conv_final_fusion2,'Shape conv_final_fusion2')
    print("\n")
    
    return conv_final_fusion2

In [5]:
#learning_rate = 0.01
#decay_rate = 1.0
#decay_steps = 1000
#num_epochs = 1
batch_size = 152
num_input = 57600
#filename = 'C:\\Users\\rober\\Practicas\\training\\training_v2.tfrecords'
#checkpoint = 'C:\\Users\\rober\\Practicas\\training\\checkpoint'
X = tf.placeholder(tf.float32, [None, num_input])

In [6]:
def run_training():
    g = tf.Graph()
    with g.as_default():
        #images,labels = inputs(batch_size,num_epochs,filename)
        #print_tensor_shape(images,'Images Despues de la funcion inputs')
        #print_tensor_shape(labels,'Labels Despues de la funcion inputs')
        #results = inference(images)
        net(X,weights,biases,dropout)

In [7]:
def main(_):
    run_training()

In [8]:
if __name__ == '__main__':
    tf.app.run

In [9]:
main(_)

X antes del Reshaping (?, 57600)
X despues del Reshaping (?, 240, 240, 4)


Shape conv1 (?, 240, 240, 64)
Shape pool1 (?, 240, 240, 64)
Shape conv2 (?, 240, 240, 64)
Shape pool2 (?, 240, 240, 64)
Shape conv3 (?, 240, 240, 64)
Shape pool3 (?, 240, 240, 64)
Shape conv4 (?, 240, 240, 64)
Shape pool4 (?, 240, 240, 64)
Shape conv5 (?, 240, 240, 64)
Shape pool5 (?, 240, 240, 64)


Shape conv6 (?, 240, 240, 128)
Shape pool4 (?, 240, 240, 128)
Shape conv7 (?, 240, 240, 128)
Shape pool7 (?, 240, 240, 128)
Shape conv8 (?, 240, 240, 128)
Shape pool8 (?, 240, 240, 128)
Shape conv9 (?, 240, 240, 128)
Shape pool9 (?, 240, 240, 128)
Shape conv10 (?, 240, 240, 128)
Shape pool10 (?, 240, 240, 128)


Shape conv11 (?, 240, 240, 256)
Shape pool11 (?, 240, 240, 256)
Shape conv12 (?, 240, 240, 256)
Shape pool12 (?, 240, 240, 256)
Shape conv13 (?, 240, 240, 256)
Shape pool13 (?, 240, 240, 256)
Shape conv14 (?, 240, 240, 256)
Shape pool14 (?, 240, 240, 256)
Shape conv15 (?, 240, 240, 256)
Shape pool15 (?, 240